In [1]:
import numpy as np
import pandas as pd
from parse import parse
import re
from aocd import get_data

In [2]:
data = get_data(year=2018, day=13)

In [3]:
test = r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """

In [202]:
def get_turn_map(n):

    def plus(d, m):
        return -(-1j) ** (1 + m)
    
    def straight(d, m):
        return 1
    
    def fwd_slash(d, m):
        return -1j * d ** 2
    
    def back_slash(d, m):
        return -fwd_slash(d, m)

    return {
        '-': straight,
        '|': straight,
        '\\': back_slash,
        '/': fwd_slash,
        '+': plus
    }

In [187]:
def render(g, l, d):
    g = g.copy()
    l_ = lambda p: (int(p.real), int(p.imag))
    d_ = {1j: '>', -1j: '<', 1: 'v', -1: '^'}.get
    for _l, _d in zip(l, d):
        i, j = l_(_l)
        g[i, j] = d_(_d)
    
    return '\n'.join([*map(''.join, g)])

def collide(g, i, j):
    a = np.array([[x for x in row] for row in g.splitlines()])
    a[i, j] = 'X'
    return '\n'.join([*map(''.join, a)])

In [275]:
def part1(data):

    right = 0 + 1j
    left = 0 - 1j
    up = -1 + 0j
    down = 1 + 0j
    
    a = np.array([[x for x in row] for row in data.splitlines()])
    
    condlist = [a == '>', a == '<', a == '^', a == 'v']
    cart_grid = np.select(condlist, [right, left, up, down])
    grid = np.select(condlist, [*'--||'], a)
    i, j = np.where(cart_grid)

    cart_loc = i + 1j * j
    cart_dir = cart_grid[i, j]
    num_cart = len(cart_loc)
    
    state = np.column_stack([
        cart_loc, cart_dir, np.zeros(num_cart)
    ])
    
    turn_map = get_turn_map(len(cart_loc))

    collision = False
    count = 0
    while not collision:
        arg = np.lexsort([state[:, 0].imag, state[:, 0].real])
        state = state[arg]
        for k in range(num_cart):
            state[k, 0] += state[k, 1]
            p, d = state[k, [0, 1]]
            x = int(p.imag)
            y = int(p.real)
            m = int(state[k, 2].real)
            g = grid[y, x]
            state[k, 2] = (m + (g == '+')) % 3
            state[k, 1] *= turn_map[g](d, m)

            if (p == state[:, 0]).sum() > 1:
                print(f"{x},{y}\nk={k}: x={x}: y={y}: dir={d}: count={count}")
#                 print(collide(render(grid, state[:, 0], state[:, 1]), y, x))
                collision = True
                break
                
        count += 1

In [276]:
part1(test)

7,3
k=1: x=7: y=3: dir=(-1+0j): count=13


In [277]:
part1(data)

57,104
k=8: x=57: y=104: dir=(1+0j): count=150


In [220]:
test2 = r"""/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/"""

In [267]:
def part2(data):

    right = 0 + 1j
    left = 0 - 1j
    up = -1 + 0j
    down = 1 + 0j
    
    a = np.array([[x for x in row] for row in data.splitlines()])
    condlist = [a == '>', a == '<', a == '^', a == 'v']
    cart_grid = np.select(condlist, [right, left, up, down])
    grid = np.select(condlist, [*'--||'], a)
    i, j = np.where(cart_grid)

    cart_loc = i + 1j * j
    cart_dir = cart_grid[i, j]
    num_cart = len(cart_loc)
    
    state = np.column_stack([
        cart_loc, cart_dir, np.zeros(num_cart)
    ])
    
    turn_map = get_turn_map(len(cart_loc))

    count = 0
    while num_cart > 1:
        arg = np.lexsort([state[:, 0].imag, state[:, 0].real])
        state = state[arg]
        mask = np.ones(num_cart, bool)
        for k in range(num_cart):
            if not mask[k]:
                continue
            state[k, 0] += state[k, 1]
            p, d = state[k, [0, 1]]
            x = int(p.imag)
            y = int(p.real)
            m = int(state[k, 2].real)
            g = grid[y, x]
            state[k, 2] = (m + (g == '+')) % 3
            state[k, 1] *= turn_map[g](d, m)

            this_mask = state[:, 0] == p
            if this_mask.sum() > 1:
                mask &= ~this_mask
                            
        state = state[mask]
        num_cart = len(state)
#         print(render(grid, state[:, 0], state[:, 1]), end='\n\n')
                
        count += 1
    
    print(f"{state[0, 0].imag:.0f},{state[0, 0].real:.0f}")
#     print(render(grid, state[:, 0], state[:, 1]))
    

In [268]:
part2(test2)

6,4


In [269]:
part2(data)

67,74
